# Environment Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!nvidia-smi

In [ ]:
!nvidia-smi

#### Download PaddlePaddle from this [website](https://www.paddlepaddle.org.cn/en/install/quick?docurl=/documentation/docs/en/install/pip/linux-pip_en.html)

In [ ]:
# Using GPU for this project - Linux
!python3 -m pip install paddlepaddle-gpu==2.4.2.post112 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html

In [ ]:
!python -m pip install paddlepaddle==2.4.2 -i https://mirror.baidu.com/pypi/simple

In [ ]:
# Check if paddle installed properly
import paddle
paddle.utils.run_check()

In [ ]:
# Colne Paddle Github
!git clone https://github.com/PaddlePaddle/PaddleOCR.git

In [ ]:
!ls 

In [ ]:
# Copy "PaddleOCR" folder which downloaded from github to Google Drive
!cp -r PaddleOCR/ /content/drive/MyDrive/MachineLearning/

### Restart Kernel and Run the scrip

### Verify installation
After the installation is complete, you can use python3 to enter the Python interpreter and then use import paddle and paddle.utils.run_check()

If PaddlePaddle is installed successfully! appears, to verify that the installation was successful.

# Prequisites

In [ ]:
import gdown

url = "https://drive.google.com/uc?id=19kSj_GswccuXk45yH87jTXLqpLTxaS67"
output = "PPOCRdata.tar"

gdown.download(url, output)

In [ ]:
!tar -xf PPOCRdata.tar && rm -rf PPOCRdata.tar

In [ ]:
!ls -lh

In [ ]:
!pip install tqdm imutils opencv-python matplotlib gdown

In [ ]:
import cv2 
import os 
#from imutils import perspective
import numpy as np 
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
from imutils import perspective

In [ ]:
!pip install --upgrade scipy numpy

# Detector Training

In [ ]:
print("Number of Training images are", len(os.listdir("/content/drive/MyDrive/MachineLearning/Total-Text")))

In [ ]:
#@title
# Code JL - Preparring Training Annotation Text file
count =0
flag = 1
train_dir = os.path.join("Total-Text", "Train")
annotation_polygonal_dir = os.path.join("Total-Text", "Annotation", "groundtruth_polygonal_annotation", "Train")
for i in tqdm(os.listdir(train_dir)):
    img_path = os.path.join(train_dir, i)
    bounding_box_path = os.path.join(annotation_polygonal_dir, "poly_gt_"+i.split('.')[0]+".txt")
    bounding_box_data = open(bounding_box_path, 'r')
    bounding_box_data = bounding_box_data.readlines()
    lis = []
    for j in bounding_box_data:
        try:
            j = j.replace('\n', '')
            x_coordinates = j.split(",")[0].split("[")[2].split("]")[0].split(" ")
            y_coordinates = j.split(",")[1].split("[")[2].split("]")[0].split(" ")
            # if len(x_coordinates)!=6 or len(y_coordinates)!=6:
            #     continue
            # if(j.split(" ")[-1]=="[]"):
            #     continue
            text = j.split(" ")[-1].split("'")[1]
            if('#' in text):
                text = '###'
            x1, x2, x3, x4, x5, x6 = x_coordinates[0], x_coordinates[1], x_coordinates[2], x_coordinates[3], x_coordinates[4], x_coordinates[5]
            y1, y2, y3, y4, y5, y6 = y_coordinates[0], y_coordinates[1], y_coordinates[2], y_coordinates[3], y_coordinates[4], y_coordinates[5]
            dic = {}
            dic["points"] = [[x1, y1], [x2, y2], [x3, y3], [x4, y4], [x5, y5], [x6, x6]]
            dic["transcription"] = str(text)
            lis.append(dic)
        except:
            flag = 0
    lis = json.dumps(lis)
    # if(flag==0):
    #     flag = 1
    #     continue
    # if(lis==[] or lis==[]):
    #     continue
    with open("train.txt", 'a') as ob:
        ob.write(img_path+"\t"+str(lis)+"\n")
    count = count + 1
print("Total number of Annotations created for Training are ", count)

In [ ]:
#@title
# Preparing Training Annotation Text file

count = 0
flag = 1
for i in tqdm(os.listdir("Total-Text/Train")):
  img_path = os.path.join("Total-Text/Train", i)
  bounding_box_path = os.path.join("Total-Text/Annotation/groundtruth_polygonal_annotation/Train", "poly_gt_"+i.split(".")[0]+".txt")
  var = open(bounding_box_path, "r")
  var = var.readlines()
  lis = []
  for j in var:
    try:
      j = j.replace("\n", '')
      x_coordinates = j.split(",")[0].split("[")[2].split("]")[0].split(" ")
      y_coordinates = j.split(",")[1].split("[")[2].split("]")[0].split(" ")
      if(len(x_coordinates) != 4 or len(y_coordinates)!=4):
        continue
      #text = j.split(",")[3].split("[")[1].split("'")[1]
      if(j.split(" ")[-1]=="[]"):
        continue
      text = j.split(" ")[-1].split("'")[1]
      if("#" in text):
        text = "###"
      x1, x2, x3, x4 = x_coordinates[0], x_coordinates[1], x_coordinates[2], x_coordinates[3]
      y1, y2, y3, y4 = y_coordinates[0], y_coordinates[1], y_coordinates[2], y_coordinates[3] 
      dic = {}
      dic["points"] = [[x1,y1], [x2,y2], [x3,y3], [x4,y4]]  
      dic["transcription"] = str(text)
      lis.append(dic)
    except:
      flag = 0
  lis = json.dumps(lis)
  if(flag == 0):
    flag = 1
    continue
  if(lis == [] or lis=="[]"):
    continue
  with open("train.txt", "a") as ob:
    ob.write(img_path+"\t"+str(lis)+"\n")
  count = count + 1

print("Total number of Annotations created for Training are ", count)

In [ ]:
#@title
# Code JL - Preparring Test or Validation Annotation Text file
count =0
flag = 1
test_dir = os.path.join("Total-Text", "Test")
annotation_polygonal_dir = os.path.join("Total-Text", "Annotation", "groundtruth_polygonal_annotation", "Test")
for i in tqdm(os.listdir(test_dir)):
    img_path = os.path.join(test_dir, i)
    bounding_box_path = os.path.join(annotation_polygonal_dir, "poly_gt_"+i.split('.')[0]+".txt")
    bounding_box_data = open(bounding_box_path, 'r')
    bounding_box_data = bounding_box_data.readlines()
    lis = []
    for j in bounding_box_data:
        try:
            j = j.replace('\n', '')
            x_coordinates = j.split(",")[0].split("[")[2].split("]")[0].split(" ")
            y_coordinates = j.split(",")[1].split("[")[2].split("]")[0].split(" ")
            # if len(x_coordinates)!=6 or len(y_coordinates)!=6:
            #     continue
            # if(j.split(" ")[-1]=="[]"):
            #     continue
            text = j.split(" ")[-1].split("'")[1]
            if('#' in text):
                text = '###'
            x1, x2, x3, x4, x5, x6 = x_coordinates[0], x_coordinates[1], x_coordinates[2], x_coordinates[3], x_coordinates[4], x_coordinates[5]
            y1, y2, y3, y4, y5, y6 = y_coordinates[0], y_coordinates[1], y_coordinates[2], y_coordinates[3], y_coordinates[4], y_coordinates[5]
            dic = {}
            dic["points"] = [[x1, y1], [x2, y2], [x3, y3], [x4, y4], [x5, y5], [x6, x6]]
            dic["transcription"] = str(text)
            lis.append(dic)
        except:
            flag = 0
    lis = json.dumps(lis)
    # if(flag==0):
    #     flag = 1
    #     continue
    # if(lis==[] or lis==[]):
    #     continue
    with open("test.txt", 'a') as ob:
        ob.write(img_path+"\t"+str(lis)+"\n")
    count = count + 1
print("Total number of Annotations created for Testing are ", count)

In [ ]:
#@title
# Preparing Testing/Validation Annotation Text file

count = 0
flag = 1
for i in tqdm(os.listdir("Total-Text/Test")):
  img_path = os.path.join("Total-Text/Test", i)
  bounding_box_path = os.path.join("Total-Text/Annotation/groundtruth_polygonal_annotation/Test", "poly_gt_"+i.split(".")[0]+".txt")
  var = open(bounding_box_path, "r")
  var = var.readlines()
  lis = []
  for j in var:
    try:
      j = j.replace("\n", '')
      x_coordinates = j.split(",")[0].split("[")[2].split("]")[0].split(" ")
      y_coordinates = j.split(",")[1].split("[")[2].split("]")[0].split(" ")
      if(len(x_coordinates) != 4 or len(y_coordinates)!=4):
        continue
      #text = j.split(",")[3].split("[")[1].split("'")[1]
      if(j.split(" ")[-1]=="[]"):
        continue
      text = j.split(" ")[-1].split("'")[1]
      if("#" in text):
        text = "###"
      x1, x2, x3, x4 = x_coordinates[0], x_coordinates[1], x_coordinates[2], x_coordinates[3]
      y1, y2, y3, y4 = y_coordinates[0], y_coordinates[1], y_coordinates[2], y_coordinates[3] 
      dic = {}
      dic["points"] = [[x1,y1], [x2,y2], [x3,y3], [x4,y4]]  
      dic["transcription"] = str(text)
      lis.append(dic)
    except:
      flag = 0
  lis = json.dumps(lis)
  if(flag == 0):
    flag = 1
    continue
  if(lis == [] or lis=="[]"):
    continue
  with open("test.txt", "a") as ob:
    ob.write(img_path+"\t"+str(lis)+"\n")
  count = count + 1

print("Total number of Annotations created for Test/Eval are ", count)

In [ ]:
#@title
!git clone https://github.com/PaddlePaddle/PaddleOCR.git

In [ ]:
%cd ./PaddleOCR

In [ ]:
!pip install -r requirements.txt

In [ ]:
%cd ..

In [ ]:
!ls

In [ ]:
!cat en_PP-OCRv3_rec.yml

In [ ]:
!wget -P ./pretrain_models/ https://paddleocr.bj.bcebos.com/pretrained/MobileNetV3_large_x0_5_pretrained.pdparams

In [ ]:
#@title
#Multi-GPU Training
!python3 -m paddle.distributed.launch --gpus '0,1'  tools/train.py -c configs/det/det_mv3_db.yml -o Global.pretrained_model=./pretrain_models/MobileNetV3_large_x0_5_pretrained

In [ ]:
#@title
#Single GPU Training
!python3 tools/train.py -c configs/det/det_mv3_db.yml  \
         -o Global.pretrained_model=./pretrain_models/MobileNetV3_large_x0_5_pretrained 

In [ ]:
#Multi-GPU Training
!python3 -m paddle.distributed.launch --gpus '0,1'  ./PaddleOCR/tools/train.py -c ./det_mv3_db.yml -o Global.pretrained_model=./pretrain_models/MobileNetV3_large_x0_5_pretrained

In [ ]:
#Single GPU Training
!python3 ./PaddleOCR/tools/train.py -c ./det_mv3_db.yml  -o Global.pretrained_model=./pretrain_models/MobileNetV3_large_x0_5_pretrained.pdparams

# Recognizer Training

In [ ]:
#@title
# Preparing the Training Annotation File and images

count = 0
flag = 1
img_counter = 0
for i in tqdm(os.listdir("Total-Text/Train")):
  img_path = os.path.join("Total-Text/Train", i)
  bounding_box_path = os.path.join("Total-Text/Annotation/groundtruth_polygonal_annotation/Train", "poly_gt_"+i.split(".")[0]+".txt")
  var = open(bounding_box_path, "r")
  var = var.readlines()
  for j in var:
    try:
      j = j.replace("\n", '')
      x_coordinates = j.split(",")[0].split("[")[2].split("]")[0].split(" ")
      y_coordinates = j.split(",")[1].split("[")[2].split("]")[0].split(" ")
      if(len(x_coordinates) != 4 or len(y_coordinates)!=4):
        continue
      #text = j.split(",")[3].split("[")[1].split("'")[1]
      if(j.split(" ")[-1]=="[]"):
        continue
      text = j.split(" ")[-1].split("'")[1]
      if("#" in text):
        text = "###"
      x1, x2, x3, x4 = x_coordinates[0], x_coordinates[1], x_coordinates[2], x_coordinates[3]
      y1, y2, y3, y4 = y_coordinates[0], y_coordinates[1], y_coordinates[2], y_coordinates[3] 
      dic = {}
      dic["points"] = [[x1,y1], [x2,y2], [x3,y3], [x4,y4]]  
      dic["transcription"] = str(text)
      lis.append(dic)
      x1,y1,x2,y2,x3,y3,x4,y4 = int(x1), int(y1), int(x2), int(y2), int(x3), int(y3), int(x4), int(x4)
      pts = np.asarray([(x1,y1), (x2,y2), (x3,y3), (x4,y4)])
      img = cv2.imread(img_path)
      warped = perspective.four_point_transform(img, pts)
      new_img_path = "crnn_train/img_"+str(img_counter)+".png"
      cv2.imwrite(new_img_path, warped)
      with open("rec_train.txt", "a") as ob:
        ob.write(new_img_path+"\t"+text+"\n")
      img_counter = img_counter + 1
    except:
      flag = 0
  if(flag == 0):
    flag = 1
    continue
  count = count + 1
print("Total number of Annotations created for Training are ", count)

In [ ]:
#@title
# Preparing the Testing/Eval Annotation File and images

count = 0
flag = 1
img_counter = 0
for i in tqdm(os.listdir("Total-Text/Test")):
  img_path = os.path.join("Total-Text/Test", i)
  bounding_box_path = os.path.join("Total-Text/Annotation/groundtruth_polygonal_annotation/Test", "poly_gt_"+i.split(".")[0]+".txt")
  var = open(bounding_box_path, "r")
  var = var.readlines()
  for j in var:
    try:
      j = j.replace("\n", '')
      x_coordinates = j.split(",")[0].split("[")[2].split("]")[0].split(" ")
      y_coordinates = j.split(",")[1].split("[")[2].split("]")[0].split(" ")
      if(len(x_coordinates) != 4 or len(y_coordinates)!=4):
        continue
      #text = j.split(",")[3].split("[")[1].split("'")[1]
      if(j.split(" ")[-1]=="[]"):
        continue
      text = j.split(" ")[-1].split("'")[1]
      if("#" in text):
        text = "###"
      x1, x2, x3, x4 = x_coordinates[0], x_coordinates[1], x_coordinates[2], x_coordinates[3]
      y1, y2, y3, y4 = y_coordinates[0], y_coordinates[1], y_coordinates[2], y_coordinates[3] 
      dic = {}
      dic["points"] = [[x1,y1], [x2,y2], [x3,y3], [x4,y4]]  
      dic["transcription"] = str(text)
      lis.append(dic)
      x1,y1,x2,y2,x3,y3,x4,y4 = int(x1), int(y1), int(x2), int(y2), int(x3), int(y3), int(x4), int(x4)
      pts = np.asarray([(x1,y1), (x2,y2), (x3,y3), (x4,y4)])
      img = cv2.imread(img_path)
      warped = perspective.four_point_transform(img, pts)
      new_img_path = "crnn_test/img_"+str(img_counter)+".png"
      cv2.imwrite(new_img_path, warped)
      with open("rec_test.txt", "a") as ob:
        ob.write(new_img_path+"\t"+text+"\n")
      img_counter = img_counter + 1
    except:
      flag = 0
  if(flag == 0):
    flag = 1
    continue
  count = count + 1
print("Total number of Annotations created for Test/Eval are ", count)

In [ ]:
%cd PaddleOCR

In [ ]:
# Download the pre-trained model of en_PP-OCRv3
!wget -P ./pretrain_models/ https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_train.tar
# Decompress model parameters
%cd pretrain_models
!tar -xf en_PP-OCRv3_rec_train.tar && rm -rf en_PP-OCRv3_rec_train.tar
%cd ..

In [ ]:
#@title
!python3 -m paddle.distributed.launch --gpus '0,1'  tools/train.py -c configs/rec/PP-OCRv3/en_PP-OCRv3_rec.yml -o Global.pretrained_model=en_PP-OCRv3_rec_train/best_accuracy

In [ ]:
#@title
!python3 -m paddle.distributed.launch --gpus '0,1'  ./PaddleOCR/tools/train.py -c ./en_PP-OCRv3_rec.yml -o Global.pretrained_model=./pretrain_models/en_PP-OCRv3_rec_train/best_accuracy

In [ ]:
#Single GPU Training
!python3 ./PaddleOCR/tools/train.py -c ./en_PP-OCRv3_rec.yml  -o Global.pretrained_model=./pretrain_models/en_PP-OCRv3_rec_train/best_accuracy.pdparams

# Converting Trained Weights to Inference Weights

In [ ]:
%cd PaddleOCR

In [ ]:
#Conversion of Recognizer Trained weights to Inference

!python3 tools/export_model.py -c configs/rec/PP-OCRv3/en_PP-OCRv3_rec.yml -o Global.pretrained_model=output/v3_en_mobile/best_accuracy  Global.save_inference_dir=./inference/en_PP-OCRv3_rec/

In [ ]:
#conversion of Detection Trained Weights to Inference

!python3 tools/export_model.py -c /content/PaddleOCR/configs/det/det_mv3_db.yml -o Global.pretrained_model="./output/db_mv3/best_accuracy" Global.save_inference_dir="./output/det_db_inference/"

# Inference using Custom Weights in PyPi Version

In [ ]:
from paddleocr import PaddleOCR,draw_ocr
from PIL import Image

In [ ]:
#Loading the detector and ocr with the previously trained weights
custom_ocr = PaddleOCR(use_angle_cls=True,
                rec_model_dir='PaddleOCR/inference/en_PP-OCRv3_rec',
                det_model_dir='/content/PaddleOCR/output/det_db_inference', 
                rec_char_dict_path='ppocr/utils/en_dict.txt',
                use_gpu=True,
                show_log=False)

In [ ]:
ocr = PaddleOCR(use_angle_cls=True, lang="en")

In [ ]:
img_path = "/content/drive/MyDrive/MachineLearning/Total-Text/Sample/Train/11664.jpg"
img = cv2.imread(img_path)
plt.imshow(img)

In [ ]:
result = custom_ocr.ocr(img)

In [ ]:
# draw result
image = Image.open(img_path).convert('RGB')
boxes = [line[0] for line in result]
txts = [line[1][0] for line in result]
scores = [line[1][1] for line in result]
im_show = draw_ocr(image, boxes, txts, scores, font_path='doc/fonts/simfang.ttf')
im_show = Image.fromarray(im_show)
im_show.save('result.jpg')

In [ ]:
im_show